# TMA4215 - Assignment 1, LU-factorisation with pivoting


**Deadline:** Wednesday September 2nd, 11:59PM.

**Requirements for approval:** You need to do two things: 1) Upload a Jupyter Notebook on Blackboard that contains the complete solution to the assigment and 2) Answer the Control question form of the assignment, you should make sure that at least some of the answers are correct.

**Supervision** of this assignment is digital. On Thursday August 27, 1600- the TA will be available for answering questions asked on Piazza and for one-to-one video conferencing on the platform Whereby.com.
At any other time you may ask questions on Piazza, but please be a little patient then, it may take some time before your question is answered.


This assignment is our way of compensating for not lecturing Gaussian elimination and LU-factorisation with pivoting. Many of you have seen this algorithm before. If your background on this method is weak, you can read up on chapters 3.2-3.5 in the text book.

In this assignment you are to implement and test a Python-program for doing LU-factorisation with pivoting.
In large simulations in real life it is often recommended to use as little memory as possible, and also avoid unnecessary operations that are time consuming. From this point of departure we would like you to make a code that takes the coefficient matrix $A$ as input and returns a representation of the matrices $P$, $L$, and $U$ which solve the equation

$$
     PA = LU
$$

where $P$ is a permutaton matrix, $L$ is a lower-triangular matrix with unit diagonal, and $U$ is an upper-triangular matrix. We *represent* the matrices in question as follows: The permutation matrix $P$ is $n\times n$, but is represented as a vector  $\mathtt{P}$ such that row number $k$ in $P$ is the canonical unit vector $e_{\mathtt{P}_k}$. Let us illustrate this by an example

$$
\mathtt{P}=
\left[
\begin{array}{r} 3 \\ 1 \\ 2 \end{array}
\right]\quad\Rightarrow\quad
P=\left[
\begin{array}{ccc}
0 & 0 & 1 \\ 1 & 0 & 0 \\ 0 & 1 & 0
\end{array}
\right]
$$

We stipulate that a Python function takes a two-dimensional numpy-array $\mathtt{A}$ as input, and returns
an *over-written* $\mathtt{A}$ which contains $L$ and $U$ in the following sense upon return:

$$
\begin{array}{ll}
\mathtt{A}[\mathtt{P}[i],j] = L_{ij} & \text{for}\ i<j \\
\mathtt{A}[\mathtt{P}[i],j] = U_{ij} & \text{for}\ i\geq j
\end{array}
$$

That $L$ has 1 on the diagonal is always the case, so the diagonal of $L$ needs not be stored. The remaining elements of $L$ and $U$ are zero and need not be stored either. The algorithm can be formulated as follows (compare to text book):


- Input: $A$ of size $n\times n$
- Initialisation
    * Let $P_i = i,\ i=0,\ldots,n-1$ be a vector (array) with $n$ components
- for $k$ **in** range(n-1):
    1. Find index $P_\ell$ such that $|\mathtt{A}_{P_\ell,k}|=\max_{k\leq i \leq n-1} |\mathtt{A}_{P_i,k}|$, i.e. scan column $k$ from the diagonal and down for the largest element in absolute value. 
    2. Swap $P_k$ by $P_\ell$.
    3. Find multipliers $A_{P_i,k}\leftarrow \frac{A_{P_i,k}}{A_{P_k,k}},\ i=k+1,\ldots,n-1$.
    4. Perform elimination, i.e. $A_{P_i,j}\leftarrow A_{P_i,j}-A_{P_i,k}\cdot A_{P_k,j},\ i,j=k+1,\ldots,n-1$
- Output: A,P

**Comment:** In practice there are of course off the shelf implementations both in Python libraries and other places for solving linear systems in an optimal way with respect to accuracy and efficiency. It also happens often that many elements of the coefficient matrix are zero and that can be exploited in various different ways. The solver you make is rather general. Irrespective of whether the code you implement is standard software, it is a useful experience to have written such a program yourself at least once, so that you understand how it works, the kind of errors that may occur etc, and you will gain understanding for instance in error analysis.



**Problem 1** Write a function for LU-factorisation with row-wise pivoting as indicated above.
A template could be


    def mylu(A):
   
    
and it should return the pivot vector (permutation vector) $\mathtt{P}$, and and over-written  version of $A$. You can also choose to copy $A$ into some other matrix $\mathtt{LU}$ from the beginning using e.g. 

    LU = A.copy()

and write into and return this matrix in order to save the input matrix $A$ 

Use the algorithm described above. See hints about indexing and useful numpy-functions that can be used below.

In [46]:
import numpy as np

def mylu(A):
    n,m = A.shape
    assert(n==m)
    
    LU = np.copy(A)
    P = np.arange(n)
    for k in range(n-1):
        pivot = np.argmax(abs(LU[P[k:], k]))+k
        P[[k,pivot]] = P[[pivot,k]]
        LU[P[k+1:],k] = LU[P[k+1:],k] / LU[P[k],k]
        LU[P[k+1:],k+1:] = LU[P[k+1:],k+1:] - np.outer(LU[P[k+1:],k], LU[P[k],k+1:])
    
    return LU, P

**Problem 2** Combine your function (mylu) with the functions for forward and backward substitution given below for computing solutions to linear systems $Ax=b$. Test it out by using $A$ and $b$ from the function getAb() below.

In [47]:
def getAb():
    A=np.array([[0.3050, 0.5399, 0.9831, 0.4039, 0.1962],
                [0.2563, -0.1986, 0.7903, 0.6807, 0.5544],
                [0.7746, 0.6253, -0.1458, 0.1704,  0.5167],
                [0.4406, 0.9256, 0.4361, -0.2254, 0.7784],
                [0.4568, 0.2108, 0.6006, 0.3677, -0.8922]])
    b=np.array([0.9876,-1.231,0.0987,-0.5544,0.7712])
    return A,b

def forward_subs(LU,P,b):
    ''' Forward substitution algorithm
    Input:
        LU contains both L and U, even if only L is needed here 
        P Integer permutation vector
        b Vector with right hand side in the problem to be solved
    Output:
        c The solution to the linear lower triangular system Lc=b 
    '''
    n, m = LU.shape
    Pb = b[P]
    c = np.zeros(n)
    c[0] = Pb[0]
    for k in range(1,n):
        c[k] = Pb[k] - LU[P[k],0:k] @ c[0:k]
        
    return c

def backward_subs(LU,P,c):
    ''' Backward substitution algorithm
    Input:
        LU contains both L and U, even though just U is needed here
        P Integer permutation vector
        c Vector containing right hand side, i.e. the function solves Ux=c
    Output:
        x Solution to the linear upper triangular system Ux = c
    '''
    n,m = LU.shape
    x = np.zeros(n)
    x[n-1] = c[n-1]/LU[P[n-1],n-1]
    for k in range(n-1,0,-1):
        x[k-1] = (c[k-1]-LU[P[k-1],k:] @ x[k:])/LU[P[k-1],k-1]
        
    return x

A,b = getAb()
LU,P = mylu(A)
c = forward_subs(LU,P,b)
x = backward_subs(LU,P,c)

print("LU solution\t ", x)
print("numpy solution:\t", np.linalg.solve(A,b))

LU solution	  [-4.74452344  5.22095642 -2.29120587  5.31222591 -1.41303909]
numpy solution:	 [-4.74452344  5.22095642 -2.29120587  5.31222591 -1.41303909]


**Control question 1:** Give the permutation vector $\mathtt{P}$ from this numerical test (multiple choice)




In [48]:
P

array([2, 3, 1, 0, 4])

**Control question 2:** Give the first component of the intermediate result $c$ (where $Lc=Pb$) (multiple choice)


In [49]:
c[0]

0.0987

**Control question 3:** Give the last component in the final answer $x$ (where $Ax=b$) with the given example.

In [50]:
x[len(x)-1]

-1.4130390910442732

**Approval**
A Jupyter notebook file with code that solves Problems 1 and 2 isto be uploaded in Blackboard and you need to answer the control questions.